This notebook is to filter the emission data from the server to exclude test and ineligible participants and trips data collected outside of the data collection timeframe.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# emission data 
emission = pd.read_csv("emission_sections_0927.csv")
# emission participants with onboard and offboard dates
timeframe = pd.read_csv("emission_participants.csv")

In [ ]:
timeframe.head()

In [ ]:
len(timeframe)

In [ ]:
len(set(emission["Token"].unique()))

In [ ]:
# filter out tokens not in final token list

participant = set(list(timeframe["Participant"]))
emission = emission[emission["Token"].isin(participant)]
emission = emission.reset_index().drop(["index"], axis = 1)
emission.head()

In [ ]:
len(participant)

In [ ]:
len(set(list((emission["Token"].unique()))))

In [ ]:
#Create a new column in the emission dataset that gives the date of each trip_section
emission["day_of_trip"] = emission['start_local_dt_year'].astype(str)+ "." + emission['start_local_dt_month'].astype(str) + "." + emission['start_local_dt_day'].astype(str)

In [ ]:
emission.head()

In [ ]:
#Convert onboard/offboard dates to datetime object
timeframe["Onboard"] = pd.to_datetime(timeframe["Onboard"])
timeframe["Offboard"] = pd.to_datetime(timeframe["Offboard"])

In [ ]:
#Convert emission trip date to datetime object
emission["day_of_trip"] = pd.to_datetime(emission["day_of_trip"])

In [ ]:
emission.head()

In [ ]:
timeframe.head()

In [ ]:
#Merge two dataframe together
df = emission.merge(timeframe, how = "left", left_on = "Token", right_on = "Participant")

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
#Filter based on time
df = df[(df["day_of_trip"] >= df["Onboard"])]
df = df[(df["day_of_trip"] <= df["Offboard"])]

In [ ]:
len(df)

In [ ]:
# identify tokens with less than 4 trips (we are defining a trip as a section, or row in dataset)
df["Token"].value_counts().sort_values()[:5]

In [ ]:
# remove users with less than 4 trips
df = df[df.Token != "AHLp9w"]

In [ ]:
len(df)

In [ ]:
df.to_csv("emission_cleaned.csv")